In [1]:
%matplotlib inline
import cartopy
import cartopy.crs as ccrs
import fiona
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
import nivapy3 as nivapy
import numpy as np
import pandas as pd
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

plt.style.use("ggplot")

In [2]:
# Connect to PostGIS
eng = nivapy.da.connect_postgis()

Connection successful.


# CL Vestland: Summarise and visualise results

## 1. Combine outputs

Merge the two summary CSV generated by the two runs of notebook 03 (handling overlapping polygons).

In [3]:
# Read data
csv_path = r"../output/hoyanger_results_summary_meqpm2pyr_main_catches.csv"
df1 = pd.read_csv(csv_path)

csv_path = r"../output/hoyanger_results_summary_meqpm2pyr_sub_catches.csv"
df2 = pd.read_csv(csv_path)

df = pd.concat([df1, df2], sort=False)

# Save combined
csv_path = r"../output/hoyanger_results_summary_meqpm2pyr.csv"
df.to_csv(csv_path, index=False)

df.head()

,dataset,station_code,min_,mean_,max_,std_,count_
0,sswc_ex_1721_meqpm2pyr_50m,HOY,-38.080956,-37.833580,-36.833298,0.420881,38743
1,fab_ex_reg_id_2050bc_50m,HOY,0.000000,0.002244,3.000000,0.082016,38772
2,fab_ex_n_1721_meqpm2pyr_50m,HOY,0.000000,0.000000,0.000000,0.000000,38743
3,fab_ex_reg_id_1721_50m,HOY,0.000000,0.002244,3.000000,0.082016,38772
4,fab_ex_ns_2050bc_meqpm2pyr_50m,HOY,0.000000,0.000000,0.000000,0.000000,38743


## 2. Get catchments

In [4]:
# Get outflows
sql = (
    "SELECT * FROM niva.stations "
    "WHERE station_id IN ( "
    "  SELECT station_id FROM niva.projects_stations "
    "  WHERE project_id IN ( "
    "    SELECT project_id FROM niva.projects "
    "    WHERE project_name = 'Høyanger' "
    "    ) "
    "  ) "
)
stn_gdf = gpd.read_postgis(sql, eng)

# Reproject to ETRS89 UTM Z33N
stn_gdf = stn_gdf.to_crs("epsg:25833")

stn_gdf.head()

,station_id,station_code,station_name,aquamonitor_id,longitude,latitude,geom
0,1285,EIR,Eiriksdal,None,6.215740,61.233612,POINT (29385.372 6820523.378)
1,1286,GAU,Gautingdalselva,None,6.144167,61.241150,POINT (25681.326 6821876.913)
2,1287,HAA,Haaland,None,6.077150,61.220030,POINT (21785.963 6820029.835)
3,1288,HOY,Hoyanger,None,6.073730,61.216397,POINT (21548.133 6819652.935)


In [5]:
# Get catchments
stn_list = list(stn_gdf["station_id"].astype(str))
bind_pars = ",".join(stn_list)
sql = f"SELECT * FROM niva.catchments " f"WHERE station_id IN ({bind_pars})"
cat_gdf = gpd.read_postgis(sql, eng)

# Reproject to ETRS89 UTM Z33N
cat_gdf = cat_gdf.to_crs("epsg:25833")

# Join codes
cat_gdf = cat_gdf.merge(
    stn_gdf[["station_id", "station_code"]], how="left", on="station_id"
)
cat_gdf = cat_gdf[["station_id", "station_code", "geom"]]

cat_gdf.head()

,station_id,station_code,geom
0,1285,EIR,"MULTIPOLYGON (((37248.834 6823946.164, 37499.3..."
1,1286,GAU,"MULTIPOLYGON (((29139.510 6824275.830, 29235.2..."
2,1287,HAA,"MULTIPOLYGON (((22172.792 6822076.278, 22182.1..."
3,1288,HOY,"MULTIPOLYGON (((29139.510 6824275.830, 29235.2..."


In [6]:
# # Use small "intercatchment" for main watershed to avoid visualising overlap
# df["station_code"].replace("Sam_Tyss", "Sam_Tyss_IntCat", inplace=True)

# # Just need mean
# df = df[["dataset", "station_code", "mean"]]

# # Join chem to spatial data
# gdf = cat_gdf.merge(df, on="station_code", how="right")

# gdf

## 3. Create map

In [7]:
# def plot_map_data(gdf, ax, title, buff=2000):
#     gdf = gdf.copy()
#     gdf["mean"] = gdf["mean"].round(2)

#     xmin, ymin, xmax, ymax = gdf.total_bounds
#     ax.set_extent([xmin - buff, xmax + buff, ymin - buff, ymax + buff], crs=crs)

#     # Plot polygons
#     gdf.plot(
#         column="mean",
#         edgecolor="none",
#         cmap="plasma",
#         # alpha=0.5,
#         legend=True,
#         # categorical=True,
#         ax=ax,
#         # legend_kwds={'loc': 'lower right'},
#     )

#     # Plot outlines
#     gdf.plot(
#         facecolor="none",
#         edgecolor="k",
#         lw=1,
#         ax=ax,
#     )

#     #     ax.add_wms(
#     #         wms="https://openwms.statkart.no/skwms1/wms.topo4.graatone",
#     #         layers=["topo4graatone_WMS"],
#     #     )

#     ax.set_title(title)

### 3.1. Critical loads

In [8]:
# fig = plt.figure(figsize=(12, 12))

# # Define co-ord system
# crs = ccrs.UTM(33)

# pars = ["claoaa", "eno3", "clminn", "clmaxnoaa", "clmaxsoaa", "clmins"]
# for idx, par in enumerate(pars):
#     ds_name = f"{par}_meqpm2pyr_50m"
#     ds_gdf = gdf.query("dataset == @ds_name").copy()
#     ax = fig.add_subplot(2, 3, idx + 1, projection=crs)
#     plot_map_data(ds_gdf, ax, par.upper() + " ($meq/m^2/yr$)")

# plt.tight_layout()
# png_path = r"../output/png/critical_loads_maps.png"
# plt.savefig(png_path, dpi=200)

### 3.2. Deposition

In [9]:
# fig = plt.figure(figsize=(12, 16))

# # Define co-ord system
# crs = ccrs.UTM(33)

# pars = ["ndep_1216", "ndep_2030bc", "sdep_1216", "sdep_2030bc"]
# for idx, par in enumerate(pars):
#     ds_name = f"{par}_meqpm2pyr_50m"
#     ds_gdf = gdf.query("dataset == @ds_name")
#     ax = fig.add_subplot(2, 2, idx + 1, projection=crs)
#     plot_map_data(ds_gdf, ax, par.upper() + " ($meq/m^2/yr$)")

# plt.tight_layout()
# png_path = r"../output/png/deposition_maps.png"
# plt.savefig(png_path, dpi=200)

### 3.3. Exceedance

In [10]:
# fig = plt.figure(figsize=(12, 16))

# # Define co-ord system
# crs = ccrs.UTM(33)

# pars = ["sswc_ex_1216", "sswc_ex_2030bc", "fab_ex_ns_1216", "fab_ex_ns_2030bc"]
# for idx, par in enumerate(pars):
#     ds_name = f"{par}_meqpm2pyr_50m"
#     ds_gdf = gdf.query("dataset == @ds_name")
#     ax = fig.add_subplot(2, 2, idx + 1, projection=crs)
#     plot_map_data(ds_gdf, ax, par.upper() + " ($meq/m^2/yr$)")

# plt.tight_layout()
# png_path = r"../output/png/exceedance_maps.png"
# plt.savefig(png_path, dpi=200)